# PREFERRED ACTIVITIES GENERATION

## 1. DATA & SETTINGS

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.sparse as sp
import scipy.sparse.linalg as spln
import utils.metropolishastings as mh
from utils.state import State
import random
from joblib import Parallel, delayed, cpu_count
import matplotlib.pyplot as plt
from scipy.stats import uniform
from matplotlib.backends.backend_pdf import PdfPages
import random
import math as m
import seaborn as sns
import copy
from cryptography.fernet import Fernet
import warnings
from utils.generate_draws import generate_draws
warnings.filterwarnings("ignore") 

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
# Parameters choice

sampleSize = 5000 # Size of the database we want
att1 = 'emp'
att2 = 'slot'
att3 = 'activity_type'

In [6]:
# Importing file
df = pd.read_csv('data/slots_nb_inter_6.csv')
tim = df#.sample(n = sampleSize, random_state = 42)

#tim['slot'] = tim['slot_hist_2']

# Divide the values in the "slot" column using the explode method
tim_slots = tim.assign(slot=tim['slot'].str.strip('[]').str.split(',')).explode('slot')

tim_slots['slot'] = tim_slots['slot'].astype(int)

# Reset dataframe index
tim_slots = tim_slots.reset_index(drop=True)

# Deciding if weekdays or weekend
responder = tim_slots[(tim_slots['day'] < 6) & (tim['emp'] >= 0)]

#responder.to_csv('original_all_wd_6.csv')

responder.head()

,HHNR,activity_type,start,end,duration,age,day,emp,motif_type,motif_type_light,nb_act,slot
21,100028,0,0,540,540,61,1,2,"0,8,0","0,8,0",2,1
22,100028,0,0,540,540,61,1,2,"0,8,0","0,8,0",2,2
23,100028,0,0,540,540,61,1,2,"0,8,0","0,8,0",2,3
24,100028,0,1035,1440,405,61,1,2,"0,8,0","0,8,0",2,5
25,100028,0,1035,1440,405,61,1,2,"0,8,0","0,8,0",2,6


## 2. FUNCTIONS

In [7]:
# Function for drawing a value using inverse transform for discrete variables
def discrete_inverse_trans(prob_vec):
    U=uniform.rvs(size=1)
    try:
        if U<=prob_vec[0]:
            return 1
        else:
            for i in range(1,len(prob_vec)+1):
                if sum(prob_vec[0:i])<U and sum(prob_vec[0:i+1])>U:
                    return i+1
    except:
        return 1

In [8]:
# Creating a probability vector from the choosen row/column from contigency matrix
def transform_into_marginals_clean(row):
    marginals= []
 
    suma = sum(row)
    for i in range(len(row)):
        marginals.append((row[i]/suma))
        
    return marginals

In [9]:
# Contigency tables
def create2dconditional(column1,column2):
    data_crosstab =  pd.crosstab(column1,column2, margins = False)
    return data_crosstab

def create3dconditional(column1, column2, column3):
    data_crosstab = pd.crosstab([column1,column2],[column3], margins = False)
    return data_crosstab  

def create4dconditional(column1, column2, column3, column4):
    data_crosstab = pd.crosstab([column1,column2,column3],[column4], margins = False)
    return data_crosstab  

def createNconditional(columns):
    data_crosstab = pd.crosstab(columns[:-1],columns[-1], margins = False)
    return data_crosstab

In [10]:
def draw_att1(att2, att3):
    '''
    Draw att1
    '''
    try:
        index = discrete_inverse_trans(transform_into_marginals_clean(cond_att1.loc[att2].loc[att3].tolist()))
        unique = sorted(cond_att1.loc[att2].loc[att3].index)  
        return unique[index-1] 
    except:
        cond_size_temp = create2dconditional(responder[att2], responder[att1])
        index = discrete_inverse_trans(transform_into_marginals_clean(cond_size_temp.loc[att2].tolist()))
        unique = sorted(cond_size_temp.loc[att2].index)          
        return unique[index-1]          

In [11]:
def draw_att2(att1, att3):
    '''
    Draw att2
    '''
    try:
        index = discrete_inverse_trans(transform_into_marginals_clean(cond_att2.loc[att1].loc[att3].tolist()))
        unique = sorted(cond_att2.loc[att1].loc[att3].index)  
        return unique[index-1] 
    except:
        cond_age_temp = create2dconditional(responder[att1], responder[att2])
        index = discrete_inverse_trans(transform_into_marginals_clean(cond_size_temp.loc[att1].tolist()))
        unique = sorted(cond_age_temp.loc[att1].index)          
        return unique[index-1]        

In [12]:
def draw_att3(att1, att2):
    '''
    Draw att3
    '''
    try:
        index = discrete_inverse_trans(transform_into_marginals_clean(cond_att3.loc[att1].loc[att2].tolist()))
        unique = sorted(cond_att3.loc[att1].loc[att2].index)  
        return unique[index-1]  
    except:
        cond_gender_temp = create2dconditional(responder[att2], responder[att3])
        index = discrete_inverse_trans(transform_into_marginals_clean(cond_gender_temp.loc[att2].tolist()))
        unique = sorted(cond_gender_temp.loc[att1].index)          
        return unique[index-1]            

In [13]:
def take_every_nth_element(draws, nb_dim, nbchain):
    data = []
    for i in range(nb_dim):
        data.append(draws[i][::nbchain])
    res = pd.DataFrame(data) 
    return res.T

## 3. GIBBS SAMPLER GENERATOR

In [14]:
class MetropolisHastingsIterator:
    """Implements the Markov chain for the MH algorithm"""

    def __init__(self, initialState):
        """Constructor

        :param initialState: the initial state of the Markov processe
        :type initialState: State

        :param userProcess: function that takes a state as input, and
            returns a new state, the forward and the backward
            transition probabilities.
        :type userProcess: state, pij, pji = fct(state)

        :param logweight: log of the unnormalized target sampling
            probability. Function that takes a state as input.
        :type logweight: float = fct(state)

        """

        self.currentState = initialState
        self.accepted = 0
        self.rejected = 0
        self.sequence = np.array([])

    def __iter__(self):
        """As the object is both an iterable and an iterator, it returns
        itself.
        """
        return self

    def getSuccessRate(self):
        """Computes the percentage of accepted draws

        :return: percentage of accepted draws
        :rtype: float
        """
        total = self.accepted + self.rejected
        if total == 0:
            return 0
        return float(self.accepted) / float(total)

    def __next__(self):
        """Generate the next state of the Markov process"""
       
        candidate, logqij, logqji = self.currentState.next_state()
        logwi = self.currentState.logweight()
        logwj = candidate.logweight()
        logratio = logwj + logqji - logwi - logqij
        logalpha_ij = min(logratio, 0)
        r = np.random.uniform()
        if np.log(r) < logalpha_ij:
            self.currentState = candidate
            self.accepted += 1
        else:
            self.rejected += 1
        return self.currentState


class AutoCorrelation:
    """Calculates the autocorrelations defined by formula (11.7)
    in Gelman et al.
    """

    def __init__(self, draws, var):
        """Constructor

        :param draws: m arrays or n draws
        :type draws: np.array([np.array(float)])

        :param var: estimate of the posterio variance, defined by (11.3)
        :type var: float

        """
        self.m, self.n = draws.shape
        self.draws = draws
        self.var = var
        self.reset()

    def reset(self):
        """Initialize the variables at the beginning of each loop."""
        self.t = 1
        self.last_rho = None
        self.negative_autocorrelation = False

    def __iter__(self):
        """As the object is both an iterable and an iterator, it returns
        itself.
        """
        self.reset()
        return self

    def variogram(self):
        """Calculates the variogram on p. 286 for the current value of t"""
        return (
            (self.draws[:, self.t :] - self.draws[:, : (self.n - self.t)]) ** 2
        ).sum() / (self.m * (self.n - self.t))

    def __next__(self):
        """Implements one iteration

        :return: current autocorrelation
        :rtype: float

        :raise StopIteration: when the list is exhausted or negative
            autocorrelation has been detected

        """
        if self.negative_autocorrelation:
            raise StopIteration
        if self.t >= self.n:
            raise StopIteration

        rho = 1 - self.variogram() / (2 * self.var)
        if not self.t % 2:
            if rho + self.last_rho < 0:
                self.negative_autocorrelation = True
        self.last_rho = rho
        self.t += 1
        return rho


def AnalyzeDraws(draws):
    """Calculates the potential scale reduction and the effective number
    of simulation draws. See Gelman et al. Chapter 11.

    :param draws: S x R array of draws, where S is the number of
        sequences, and R the number of draws per sequence.

    :type draws: numpy.array

    :return: potential scale reduction, effective numner of draws and
        mean of the indicators
    :rtype: float, int, numpy.array(float)
    """

    nbrOfSequences, nbrOfDraws = draws.shape
    m = 2 * nbrOfSequences
    n = int(nbrOfDraws / 2)
    draws = draws.reshape(m, n)

    # The name of the variables below refer to the notations in
    # Gelman et al.

    # Means
    phi_bar_dot_j = [np.mean(d) for d in draws]
    phi_bar = np.mean(phi_bar_dot_j)
    B = np.var(phi_bar_dot_j, ddof=1) * n

    # Variances. ddof=1 means that we divide by n-1 and not by
    # n. See numpy documentation
    s_j_squared = [np.var(d, ddof=1) for d in draws]
    W = np.mean(s_j_squared)

    # Calculation of the marginal posterior variance (11.3) and
    # the potential scale reduction (11.4)
    var_plus = (n - 1) * W / n + B / n
    R_hat = np.sqrt(var_plus / W)

    # Calculation of the effective number of simulation draws
    ac = AutoCorrelation(draws, var_plus)
    neff = m * n / (1 + 2 * sum(ac))

    return R_hat, neff, phi_bar


def MetropolisHastings(
    initialStates,
    numberOfDraws=1000,
    maxNumberOfIterations=10,
):
    """Implements the Metropolis Hastings algorithm, checking for stationarity

    :param initialStates: list of inintial states for the sequences
    :type initialStates: list(state)

    :param numberOfDraws: numberOfDraws requested by the user
    :type numberOfDraws: int

    :param maxNumberOfIterations: Draws are generated at each
        iteration until stationarity is detected. This parameter sets
        a maximum number of these iterations.
    :type maxNumberOfIterations: int

    :return: the draws, the estimated average, the status of
        stationarity and the number of iterations
    :rtype: numpy.array, float, bool, int

    """
    # number of sequences = number of initial states
    indicator = False
    m = 2 * len(initialStates)
    
    iterators = [
        MetropolisHastingsIterator(init_state) for init_state in initialStates
    ]
    
    number_of_parallel_jobs = min(cpu_count(), len(iterators))

    # Warmup
    print(
        f'***** Warmup with {len(iterators)} sequences '
        f'of {numberOfDraws} draws *****'
    )
    # We first generate draws that are not stored for the warmup
    # of the Markov processes
    _ = Parallel(n_jobs=number_of_parallel_jobs, prefer='threads')(
        delayed(generate_draws)(i, numberOfDraws) for i in iterators
    )
    


    currentNumberOfDraws = numberOfDraws
    for trials in range(maxNumberOfIterations):
        print(
            f'***** Trial {trials} with {len(iterators)} sequences '
            f'of {currentNumberOfDraws} draws *****'
        )
        # Generate the draws
        draws = Parallel(n_jobs=number_of_parallel_jobs, prefer='threads')(
            delayed(generate_draws)(i, currentNumberOfDraws) for i in iterators
        )
        
       
        #it is necessary to omit array of individuals for convergancy analysis, it should be adapted also for individuals later on
        #print('pre konverzije')
        #print(draws)
        draws = np.array(draws[0:3])
        print(f'after conversion:{draws}')
       
     
        print(f'Generated draws: {draws.shape}')
        for i in iterators:
            print(f'Success rate: {i.getSuccessRate()}')

        # The dimensions of draws are: #sequences x #draws x #indicators
        # We change it to obtain: #indicators x #sequences x #draws

        draws = np.swapaxes(np.swapaxes(draws, 0, 2), 1, 2)
        
        # Check for stationarity for each indicator
        draws_analyse = draws[0:2]
        R_hat, neff, phi_bar = zip(
            *[AnalyzeDraws(draw_per_indicator) for draw_per_indicator in draws_analyse]
        )
        R_hat = np.array(R_hat)
        neff = np.array(neff)
        target_neff = 5 * m
        phi_bar = np.array(phi_bar)

        print(f'Potential scale reduction: {R_hat}')
        print('    should be at most 1.1')
        print(f'Effective number of simulation draws: {neff}')
        print(f'    should be at least {target_neff}')
        if np.all(R_hat <= 1.1) and np.all(neff >= target_neff):
            # We merge the draws from the various sequences before
            # returning them
            final_draws = np.array([t.flatten() for t in draws])
            return final_draws, phi_bar, True, trials + 1
        min_neff = np.min(neff)
        if min_neff >= target_neff:
            inflate = 2
        else:
            inflate = 1 + int(target_neff / min_neff)
        currentNumberOfDraws = currentNumberOfDraws * inflate
    print(
        f'Warning: the maximum number ({maxNumberOfIterations}) '
        f'of iterations has been reached before stationarity.'
    )
    # We merge the draws from the various sequences before returning
    # them.
    final_draws = np.array([t.flatten() for t in draws])
    #the last one is one sequence
    return final_draws, phi_bar, False, maxNumberOfIterations


In [15]:
class distribution_draws(State):
    def __init__(
        self, 
        xy,
        
    ):
        """Constructor
        """
        self.xy = xy
       
      
    def indicators(self):
        """The indicators are the parameters of interest, generated 
        by each draw. In this case, the indicators are the states 
        themselves.

        :return: array of indicators
        :rtype: numpy.array()
        """
        #print(f'Draw:{self.xy}')
        #[htype,hsize,nbcars,individuals]
        return self.xy
    
    
    #this can be expanded to various number of dimensions
    def next_state(self):
        
        r = random.randint(0, 2) #num_dim-1

        if(r==0): #draw att1 

            att2 = self.xy[1]
            att3 = self.xy[2]            
            att1 = draw_att1(att2, att3)

        elif(r==1): #draw att2

            att1 = self.xy[0]
            att3 = self.xy[2]
            att2 = draw_att2(att1,att3)


        else: #draw att3

            att1 = self.xy[0]
            att2 = self.xy[1]
            att3 = draw_att3(att1, att2)
        
        #print(f"{hsize_value} {age_owner} {gender_owner}")
        #this function should be expanded to generate different attributes
        
        
    
        next_state = distribution_draws(
            np.array([att1, att2, att3]),
            
       )
        
       
        return (
            next_state,
            0,
            0,
        )
      
    
    def logweight(self):
         return 0

## 4. GENERATION

In [16]:
# Conditional tables
cond_att1 = create3dconditional(responder[att2],responder[att3],responder[att1])
cond_att2 = create3dconditional(responder[att1],responder[att3],responder[att2])
cond_att3 = create3dconditional(responder[att1],responder[att2],responder[att3])

In [17]:
numberOfDraws = sampleSize
#numberOfDraws = 177036
maxNumberOfIterations = 5

In [18]:
# Initial states
# It is important to choose some states that exist in dataset
# ((emp, slot, act))

initialStates = [    
    distribution_draws(        
        np.array([2, 5, 4])
    ),
    distribution_draws(        
        np.array([4, 1, 0])
    ),
    distribution_draws(        
        np.array([1, 2, 8])
    ),
    distribution_draws(        
        np.array([3, 3, 2])
    )
]

In [19]:
%%time

draws, estimates, convergence, numberOfTrials = MetropolisHastings(
    initialStates,
    numberOfDraws,
    maxNumberOfIterations,
)

***** Warmup with 4 sequences of 5000 draws *****
***** Trial 0 with 4 sequences of 5000 draws *****
after conversion:[[[2 4 0]
  [1 4 0]
  [1 4 2]
  ...
  [4 6 8]
  [4 6 0]
  [4 6 0]]

 [[2 5 2]
  [2 5 2]
  [2 5 0]
  ...
  [4 3 8]
  [4 3 5]
  [1 3 5]]

 [[1 2 2]
  [1 2 2]
  [2 2 2]
  ...
  [4 5 3]
  [4 2 3]
  [4 6 3]]]
Generated draws: (3, 5000, 3)
Success rate: 1.0
Success rate: 1.0
Success rate: 1.0
Success rate: 1.0
Potential scale reduction: [1.00488703 1.00010875]
    should be at most 1.1
Effective number of simulation draws: [1952.67519112 2686.87290526]
    should be at least 40
CPU times: total: 37.8 s
Wall time: 35.9 s


#### Skipping phase and taking n-elements

In [20]:
sample = take_every_nth_element(draws,3,3) #number of chains is always equal to number of intitial_states
sample.rename(columns = {0:att1, 1:att2, 2:att3}, inplace = True)

In [21]:
# Save the file with the values generated
#sample.to_csv('2023_05_08_prefered_activity_wd_6_full.csv')

sample

,emp,slot,activity_type
0,2,4,0
1,1,2,2
2,4,2,0
3,4,4,0
4,3,4,0
...,...,...,...
4995,3,3,0
4996,4,2,0
4997,4,1,0
4998,3,5,0
